In [1]:
#import all the dependency
#utils include some functions that can help us to manitipulate midi file, it is written by https://github.com/brannondorsey/midi-rnn
import os, argparse, time
import utils
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adam, Adamax, Nadam
from keras.layers import Conv1D,GlobalMaxPooling1D
OUTPUT_SIZE = 129 # 0-127 notes + 1 for rests

Using TensorFlow backend.


In [2]:
# all gthe paramater we may use later on 
data_dir='data/midi'
experiment_dir='experiments/default'
window_size=50
batch_size=32
num_epochs=10
dropout=0.2
grad_clip=5.0
n_jobs=1
max_files_in_ram=25
input_shape=(50,129)

In [3]:
#function to build a 2-Layer LSTM model
def LSTM_model(input_shape):
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(LSTM(128, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(129))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    model.summary()
    return model

In [4]:
#function to build a 1-D casual convolutional dilated model
def dilated_casual_1D_conv_model(input_shape):
    model = Sequential()
    filters=[128, 128, 128, 128]
    kernel_size=2
    model.add(Conv1D(filters[0],
                 kernel_size,
                 input_shape=input_shape,
                 padding='causal',
                 dilation_rate=2,
                 strides=1))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Conv1D(filters[1],
                 kernel_size,
                 padding='causal',
                 dilation_rate=4,
                 strides=1))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Conv1D(filters[2],
                 kernel_size,
                 padding='causal',
                 dilation_rate=8,
                 strides=1))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Conv1D(filters[3],
                 kernel_size,
                 padding='causal',
                 activation='relu',
                 dilation_rate=8,
                 strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(128))
    model.add(Dropout(0.2))    
    model.add(Activation('relu'))
    
    model.add(Dense(129))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', 
                  optimizer='adam',
                  metrics=['accuracy'])
    model.summary()
    return model

In [5]:
#denfine the callbacks function to monitor the trainning process
def get_callbacks(experiment_dir, checkpoint_monitor='val_acc'):
    
    callbacks = []
    
    # save model checkpoints
    filepath = os.path.join(experiment_dir, 
                            'checkpoints', 
                            'checkpoint-epoch_{epoch:03d}-val_acc_{val_acc:.3f}.hdf5')

    callbacks.append(ModelCheckpoint(filepath, 
                                     monitor=checkpoint_monitor, 
                                     verbose=1, 
                                     save_best_only=False, 
                                     mode='max'))

    callbacks.append(ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.5, 
                                       patience=3, 
                                       verbose=1, 
                                       mode='auto', 
                                       epsilon=0.0001, 
                                       cooldown=0, 
                                       min_lr=0))

    callbacks.append(TensorBoard(log_dir=os.path.join(experiment_dir, 'tensorboard-logs'), 
                                histogram_freq=0, 
                                write_graph=True, 
                                write_images=False))

    return callbacks

In [6]:
try:
    # get paths to midi files in --data_dir
    midi_files = [os.path.join(data_dir, path) \
                  for path in os.listdir(data_dir) \
                  if '.mid' in path or '.midi' in path]
#print midi_files
except OSError as e:
    print("invalid path")
#check the data
if len(midi_files) < 1:
    print("no_enough data")

In [7]:
verbose=True
experiment_dir = utils.create_experiment_dir(experiment_dir, verbose)

[*] Created experiment directory experiments\02
[*] Created checkpoint directory experiments\02\checkpoints
[*] Created log directory experiments\02\tensorboard-logs


In [8]:
#split the training and cross validation set
val_split = 0.2 # use 20 percent for validation
val_split_index = int(float(len(midi_files)) * val_split)
#define the generator to generate the data in real-time
train_generator = utils.get_data_generator(midi_files[0:val_split_index], 
                                           window_size=window_size,
                                           batch_size=batch_size,
                                           num_threads=n_jobs,
                                           max_files_in_ram=max_files_in_ram)

val_generator = utils.get_data_generator(midi_files[val_split_index:], 
                                         window_size=window_size,
                                         batch_size=batch_size,
                                         num_threads=n_jobs,
                                         max_files_in_ram=max_files_in_ram)

In [9]:
#In our experiment, we have two model, one is 2-layer LSTM, another is dilated casual 1 D convolution
#Details are included in the report
#you can commond out one of the two code below to choose another model used in the trainning

model = LSTM_model(input_shape)
print("------------------------------------------------------------------------------------")
model = dilated_casual_1D_conv_model(input_shape)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 50, 128)           132096    
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 129)               16641     
_________________________________________________________________
activation_1 (Activation)    (None, 129)               0         
Total params: 280,321.0
Trainable params: 280,321.0
Non-trainable params: 0.0
________________________________________________________________

In [ ]:
#train the model
#not enough time to rerun the model. So, no more running result later
print('fitting model...')
# this is a somewhat magic number which is the average number of length-20 windows
# calculated from ~5K MIDI files from the Lakh MIDI Dataset.
magic_number = 827
start_time = time.time()
model.fit_generator(train_generator,
                    steps_per_epoch=len(midi_files) * magic_number / batch_size, 
                    epochs=1,
                    validation_data=val_generator, 
                    validation_steps=len(midi_files) * 0.2 * magic_number / batch_size,
                    verbose=1
                   )

fitting model...
1Epoch 1/1



In [ ]:
def get_experiment_dir(experiment_dir):
    if experiment_dir == 'experiments/default':
        dirs_ = [os.path.join('experiments', d) for d in os.listdir('experiments') \
                 if os.path.isdir(os.path.join('experiments', d))]
        experiment_dir = max(dirs_, key=os.path.getmtime)

    if not os.path.exists(os.path.join(experiment_dir, 'model.json')):
        print('no model')

    return experiment_dir

In [ ]:
midi_files = [ args.prime_file ] if args.prime_file else \
             [ os.path.join(args.data_dir, f) for f in os.listdir(args.data_dir) \
             if '.mid' in f or '.midi' in f ]

experiment_dir = get_experiment_dir(args.experiment_dir)
utils.log('Using {} as --experiment_dir'.format(experiment_dir), args.verbose)

if not args.save_dir:
    args.save_dir = os.path.join(experiment_dir, 'generated')

if not os.path.isdir(args.save_dir):
    os.makedirs(args.save_dir)
    utils.log('Created directory {}'.format(args.save_dir), args.verbose)

model = dilated_casual_1D_conv_model(input_shape)

window_size = model.layers[0].get_input_shape_at(0)[1]
seed_generator = utils.get_data_generator(midi_files, 
                                          window_size=window_size,
                                          batch_size=32,
                                          num_threads=1,
                                          max_files_in_ram=10)


In [ ]:
# generate 10 tracks using random seeds
utils.log('Loading seed files...', args.verbose)
X, y = seed_generator.next()
generated = utils.generate(model, X, window_size, 
                           args.file_length, args.num_files, args.midi_instrument)
# print(len(generated[0]))
# #print(np.sum(generated,axis=1))
for i, midi in enumerate(generated):
    file = os.path.join(args.save_dir, '{}.mid'.format(i + 1))
    midi.write(file.format(i + 1))
    utils.log('wrote midi file to {}'.format(file), True)